# Iniciando o projeto com as importação das dependencias


### FASE 1 - CONFIGURAÇÃO


In [5]:
# imports do projeto

import pandas as pd
import gdown

### FUNÇÕES


In [2]:
def montar_estatisticas_dataFrame(originial, tratado):
    total_registros = len(originial)
    total_tratado = len(tratado)
    dados_invalidos = total_registros - total_tratado
    dados_validos = total_registros - dados_invalidos
    percentual_erro = dados_invalidos / total_registros

    # Exibir as métricas
    print(f"📊 Total de registros importados: {total_registros}")*+
    print(f"✅ Registros válidos: {dados_validos}")
    print(f"❌ Registros com erro: {dados_invalidos}")
    print(f"% Registros com erro: {percentual_erro:.2f}%")

### FASE 2 - INGESTÃO


In [ ]:
# importando o dataframe
path_url = 'https://drive.google.com/uc?export=download&id=1Vumu8jo3P3umuUtBZb6mn7YoVIo4X0ON'
path_local = "./dados/df_fraud_credit.csv"
gdown.download(path_url, path_local, quiet=False)

In [ ]:
df = pd.read_csv(path_local)
parquet_file = "./dados/df_fraud_credit.parquet"
df.to_parquet(parquet_file, engine="pyarrow", index=False)

In [12]:
df_parquet = pd.read_parquet(parquet_file, engine="pyarrow")

### FASE 3 - TRATAMENTO DOS DADOS


In [ ]:
zeroFilter = df_parquet.location_region != "0"
dados_tratados = df_parquet.loc[zeroFilter]

noneRiskScore = dados_tratados.risk_score != "none"
dados_tratados = dados_tratados.loc[noneRiskScore]

noneAmount = dados_tratados.amount != "none"
dados_tratados = dados_tratados.loc[noneAmount]

In [ ]:
montar_estatisticas_dataFrame(df, dados_tratados)

In [22]:
dados_tratados["risk_score"] = pd.to_numeric(dados_tratados["risk_score"])

In [23]:
tabela_risk_score = (
    dados_tratados.groupby("location_region")["risk_score"]
    .mean()
    .reset_index()
    .sort_values(by="risk_score", ascending=False)
)

In [ ]:
print(tabela_risk_score.head())

In [25]:
dados_tratados["timestamp"] = pd.to_datetime(dados_tratados["timestamp"])
dados_tratados["amount"] = dados_tratados["amount"].values.astype("float64")

In [26]:
df_sale = dados_tratados[dados_tratados["transaction_type"] == "sale"]

In [27]:
df_recent = df_sale.sort_values(
    by=["receiving_address", "timestamp"], ascending=[True, False]
)
df_recent = df_recent.drop_duplicates(subset=["receiving_address"], keep="first")


In [28]:
top_receiving_addresses = df_recent.nlargest(3, "amount")[
    ["receiving_address", "amount", "timestamp"]
]

In [ ]:
print("\n📌 Top 3 'receiving address' com maior 'amount':")
print(top_receiving_addresses)